In [55]:
import numpy as np
import tensorflow as tf
import sklearn.metrics as met
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping

import keras

    
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)
    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value


In [56]:
# generation a small dataset
N_all = 1000
N_tr = int(0.7 * N_all)
N_te = N_all - N_tr

np.random.seed(0)
x_dim = 50
y_dim = 10
X = np.random.rand(N_all,x_dim)
y = np.random.randint(0,2,size=(N_all-1,y_dim))
y_app = np.zeros(shape= (1,y_dim))
y_app[0] = 1
y = np.vstack((y,y_app))

X_train, X_valid = X[:N_tr, :], X[N_tr:, :]
y_train, y_valid = y[:N_tr, :], y[N_tr:, :]

In [63]:
# model & train
model = Sequential()
model.add(Dense(y_dim,input_shape=(X.shape[1],)))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', auc_roc])

class MyCallBack(keras.callbacks.Callback):
    def __init__(self, callbacks,model):
            super().__init__()
            self.callback = callbacks
            self.model = model
            self.model_original = model
            self.aucs = 0

    def on_epoch_begin(self,epoch,logs=None):
            self.model = self.model_original
            self.callback.on_epoch_begin(epoch, logs=logs)

    def on_epoch_end(self,epoch,logs=None):
            self.model = self.model_original
            y_pred = self.model.predict(self.validation_data[0])
            self.aucs = met.roc_auc_score(self.validation_data[1], y_pred)
            print('\nSKLearn auc: {}\n'.format(self.aucs))
            self.callback.on_epoch_end(epoch, logs=logs)

    def on_batch_end(self, batch, logs=None):
            self.model = self.model_original
            self.callback.on_batch_end(batch, logs=logs)

    def on_batch_begin(self, batch, logs=None):
            self.model = self.model_original
            self.callback.on_batch_begin(batch, logs=logs)

    def on_train_begin(self, logs=None):
            self.model = self.model_original
            self.callback.set_model(self.model)
            self.callback.on_train_begin(logs=logs)

    def on_train_end(self, logs=None):
            self.model = self.model_original
            self.callback.on_train_end(logs=logs)

'''
cbk_tb = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, batch_size=batch_size, write_graph=True,
                                         write_grads=False, write_images=False, embeddings_freq=0,
                                         embeddings_layer_names=None, embeddings_metadata=None)
'''
cbk_es = keras.callbacks.EarlyStopping(monitor='train_loss', mode='max',
                                          min_delta=0, patience=0, verbose=1)
'''
cbk_mc = keras.callbacks.ModelCheckpoint(monitor='val_loss', mode='max', save_best_only=True, 
                                            filepath=checkpoint_dir+checkpoint_file_name, 
                                            verbose=1)
'''

#cbk = MyCallBack(cbk_tb, model)
cbk1 = MyCallBack(cbk_es, model)
#cbk2 = MyCallBack(cbk_mc, model)

callbacks = [cbk1]

In [64]:
#my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

In [65]:
model.fit(X_train, y_train, batch_size=len(X_train), validation_data=(X_valid, y_valid), epochs=5, verbose=1,callbacks=callbacks)

# # or use independent valid set
# model.fit(X_train, y_train,
#           validation_data=(X_valid, y_valid),
#           batch_size=32, nb_epoch=5, verbose=1,
#           callbacks=my_callbacks)

Train on 700 samples, validate on 300 samples
Epoch 1/5
700/700 [==============================] - 5s 7ms/step - loss: 0.7593 - acc: 0.4970 - auc_roc: 0.0000e+00 - val_loss: 0.7514 - val_acc: 0.5017 - val_auc_roc: 0.4952

SKLearn auc: 0.5074847219673992

Epoch 2/5
700/700 [==============================] - 0s 12us/step - loss: 0.7568 - acc: 0.4959 - auc_roc: 0.4987 - val_loss: 0.7488 - val_acc: 0.5013 - val_auc_roc: 0.4972

SKLearn auc: 0.5075690370899403

Epoch 3/5
700/700 [==============================] - 0s 10us/step - loss: 0.7541 - acc: 0.4983 - auc_roc: 0.4987 - val_loss: 0.7463 - val_acc: 0.5047 - val_auc_roc: 0.4977

SKLearn auc: 0.5076866824052282

Epoch 4/5
700/700 [==============================] - 0s 11us/step - loss: 0.7513 - acc: 0.5004 - auc_roc: 0.4986 - val_loss: 0.7437 - val_acc: 0.5033 - val_auc_roc: 0.4979

SKLearn auc: 0.5076752963600113

Epoch 5/5
700/700 [==============================] - 0s 11us/step - loss: 0.7486 - acc: 0.4986 - auc_roc: 0.4986 - val_loss: 0.

/home/emanuele.dallalonga/miniconda3/envs/musictagger/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `train_loss` which is not available. Available metrics are: val_loss,val_acc,val_auc_roc,loss,acc,auc_roc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [66]:
y_pred = model.predict(X_valid)

In [54]:
aucs = []
for i in range(y.shape[1]):
    fpr, tpr, thresholds = met.roc_curve(y_valid[:,i], y_pred[:,i],)
    print(len(thresholds))
    aucs.append(met.auc(fpr,tpr))
    print(aucs[i])
np.mean(aucs)

163
0.5046255114748266
150
0.5593777777777778
154
0.4983482142857143
159
0.49655540246233165
152
0.5046699875466999
145
0.45399182076813654
152
0.4644321625958282
142
0.531445842037424
147
0.489894943019943
140
0.4440625


0.49474041619686815

In [47]:
y_pred

array([[0.4304548 , 0.44790632, 0.5572338 , ..., 0.44073677, 0.8068553 ,
        0.478079  ],
       [0.47090185, 0.32741264, 0.68185157, ..., 0.5443486 , 0.6817875 ,
        0.35700214],
       [0.5274341 , 0.51642483, 0.44326785, ..., 0.5007694 , 0.6795557 ,
        0.45367467],
       ...,
       [0.5411633 , 0.47085306, 0.45827228, ..., 0.50788945, 0.682047  ,
        0.34951356],
       [0.49606347, 0.40679526, 0.58102834, ..., 0.47269872, 0.6160014 ,
        0.401137  ],
       [0.45154715, 0.39507172, 0.7008329 , ..., 0.57089204, 0.49671045,
        0.5038259 ]], dtype=float32)